In [1]:
from pydantic import BaseModel, Field
from datasets import load_dataset, Dataset
from peft import LoraConfig

import xverify as xv

In [2]:

class Tools(BaseModel):
    """
    Run a tool.
    """
    tool_use: xv.XMLToolUse[xv.calculator, xv.search] = Field(..., description="The tool call to use")


class FinalAnswer(BaseModel):
    """
    Return a final answer.
    """
    answer: str = Field(..., description="The final answer to the question")


class Reason_and_Act(BaseModel):
    scratchpad: str = Field(
        ...,
        description="Information from the Observation useful to answer the question",
    )
    reasoning: str = Field(
        ...,
        description="It describes your thoughts about the question you have been asked",
    )
    response: Tools | FinalAnswer

def tool_response_func(model: Reason_and_Act) -> dict | None:
    return xv.run_tools(model.response)

guided_schema = xv.GuidedSchema(Reason_and_Act, schema="xml", tool_response_func=tool_response_func)

In [3]:

def format_prompt(prompt: str, system_prompt: str | None = None) -> list[dict[str, str]]:
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})
    return messages

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

SYSTEM_PROMPT = f"""\
You are a helpful assistant, responding in XML structured output.

- Think step by step using the scratchpad and reasoning outputs. You have {guided_schema.max_steps - 1} steps to think before responding.
- Use the tools provided. DO NOT rely on your own knowledge when a tool is available to help you.
- Respond with a final answer only once your are absolutely sure you have the answer.

Respond with a XML object, following the schema below:

{guided_schema.doc}
"""

CODE_PROMPT = """\
Given a math problem, use step-by-step reasoning and code execution to solve the problem.

For each step:
1. Think through your reasoning inside <reasoning> tags
2. Write Python scripts inside <code> tags to work out calculations
   - Functions and variables do not persist across <code> calls and should be redefined each time
   - Scripts should be written in Python 3.10+ syntax, and should run in under 10 seconds
   - Any desired outputs should be printed using print() statements
   - You may import numpy, scipy, and sympy libraries for your calculations
3. You will see the output from print() statements in your code in <output> tags
4. Continue until you can give the final answer inside <answer> tags
"""

dataset: Dataset = load_dataset("openai/gsm8k", "main", split="train") # type: ignore
dataset = dataset.map(lambda x: {
    "prompt": format_prompt(x["question"], CODE_PROMPT),
    "answer": extract_hash_answer(x["answer"])
})

In [4]:


model_name = "Qwen/Qwen2.5-1.5B-Instruct"
model, tokenizer = xv.get_model_and_tokenizer(model_name)

run_name = "gsm8k-calculator-peft_" + model_name.split("/")[-1].lower()
training_args = xv.get_default_grpo_config(run_name, num_gpus=1)

peft_config = LoraConfig(
    r=16,
    lora_alpha=64,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj"]
)

def exact_answer_reward_func(completions, answers, **kwargs) -> list[float]:
    """Reward function that checks if the final answer matches the expected answer."""
    def _check_answer(trajectory: list[dict[str, str]], answer: int) -> float:
        """Extract the last answer from a trajectory."""
        last_message = trajectory[-1]
        assert last_message['role'] == 'assistant', "should be assistant"
        parsed: Reason_and_Act = guided_schema.parse(last_message['content']) # type: ignore
        if parsed is None or not isinstance(parsed.response, FinalAnswer):
            return 0.
        return 1. if parsed.response.answer == answer else 0.
    return [_check_answer(c, a) for c, a in zip(completions, answers)]

trainer = xv.GRPOGuidedTrainer(
    guided_schema=guided_schema,
    model=model,
    args=training_args,
    train_dataset=dataset,
    peft_config=peft_config,
    reward_funcs=[exact_answer_reward_func],
)
trainer.train()

Using Liger kernel
Applied Liger kernels to Qwen2


You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/home/tom/fun/xverify/.venv/lib/python3.12/site-packages/trl/trainer/grpo_trainer.py:471: UserWarning: The requested device cuda:0 is also being used for training. For higher throughput and to avoid out-of-memory errors, it is recommended to use a dedicated device for vLLM. If this is intentional, you may ignore this warning but should adjust `vllm_gpu_memory_utilization` accordingly.
  warnings.warn(


INFO 03-13 10:30:01 __init__.py:207] Automatically detected platform cuda.
INFO 03-13 10:30:06 config.py:549] This model supports multiple tasks: {'generate', 'embed', 'classify', 'score', 'reward'}. Defaulting to 'generate'.
INFO 03-13 10:30:06 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='Qwen/Qwen2.5-1.5B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-1.5B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda:0, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_e

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 03-13 10:30:08 model_runner.py:1115] Loading model weights took 2.8875 GB
INFO 03-13 10:30:10 worker.py:267] Memory profiling takes 1.70 seconds
INFO 03-13 10:30:10 worker.py:267] the current vLLM instance can use total_gpu_memory (23.58GiB) x gpu_memory_utilization (0.30) = 7.07GiB
INFO 03-13 10:30:10 worker.py:267] model weights take 2.89GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 2.02GiB; the rest of the memory reserved for KV Cache is 2.11GiB.
INFO 03-13 10:30:11 executor_base.py:111] # cuda blocks: 4929, # CPU blocks: 9362
INFO 03-13 10:30:11 executor_base.py:116] Maximum concurrency for 32768 tokens per request: 2.41x
INFO 03-13 10:30:14 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utili

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:15<00:00,  2.27it/s]

INFO 03-13 10:30:30 model_runner.py:1562] Graph capturing finished in 15 secs, took 0.20 GiB
INFO 03-13 10:30:30 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 21.44 seconds



wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: tompollak to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/home/tom/fun/xverify/.venv/lib/python3.12/site-packages/outlines/fsm/guide.py:110: UserWarning: Outlines' public *community-contributed* CFG structured generation is experimental. Please review https://dottxt-ai.github.io/outlines/latest/reference/generation/cfg#disclaimer
  warnings.warn(


GrammarError: Unexpected colon, at line 1 column 7

root ::= grammar-models
      ^


In [7]:
print(guided_schema.gbnf)

root ::= grammar-models
string ::= nl [^\n<] ([^<])*
boolean ::= nl "true" | nl "false"
integer ::= nl [0-9]+
float ::= nl "-"? [0-9]+ ("." [0-9]+)?
null ::= nl "null"
nl ::= "\n"
Reason_and_Act ::=  "<Reason_and_Act>" nl "<scratchpad>" string nl "</scratchpad>" nl "<reasoning>" string nl "</reasoning>" nl "<response>" string nl "</response>" nl "</Reason_and_Act>"
grammar-models ::= Reason_and_Act
